In [11]:
import sklearn
from sklearn.linear_model import Ridge
import time
from scipy.io import loadmat
import functionList
import numpy as np
import pandas as pd
import torch

In [13]:
raw = loadmat(r"C:\Users\parkf\Desktop\summer2021\eegData\for_Barry\282L.mat") #change this to try different one off files
X = raw['data']   #[0][0]['data'] #file format is different from ###L/R.mat vs the sample data
l = X.shape[0]

In [14]:
#holdStack = X
#for i in range(9):
#    X = np.hstack((X,holdStack))

In [19]:
memVec = [4, 8, 16, 20]
ridgeVec = [.1]
ridgeVecNormal = np.dot(ridgeVec,l)

s = (len(memVec),len(ridgeVec),10)
error = np.zeros(s)
runTimes = np.zeros(s)
for memNumber in memVec:
    featureMat = functionList.buildFeatureMat(X,memNumber)
    for regularizer in ridgeVecNormal: 
        RR = Ridge(alpha=regularizer)
        crossVal = sklearn.model_selection.KFold(n_splits=2)
        cvRun = 0
        for train, test in crossVal.split(featureMat):
            start = time.time()
            RR.fit(featureMat[train,:],X[train])
            end = time.time()
            pred = RR.predict(featureMat[test])
            error[memVec.index(memNumber), ridgeVec.index(regularizer/l), cvRun] = error[memVec.index(memNumber), ridgeVec.index(regularizer/l), cvRun] + functionList.findMSE(pred, X[test])
            runTimes[memVec.index(memNumber), ridgeVec.index(regularizer/l), cvRun] = end - start
            cvRun = cvRun + 1


In [23]:
print(runTimes)
print("erros below")
print(error)

[[[0.23037958 0.18749976 0.         0.         0.         0.
   0.         0.         0.         0.        ]]

 [[0.59040189 0.31218648 0.         0.         0.         0.
   0.         0.         0.         0.        ]]

 [[0.66662407 0.69508362 0.         0.         0.         0.
   0.         0.         0.         0.        ]]

 [[0.80906534 0.76701593 0.         0.         0.         0.
   0.         0.         0.         0.        ]]]
erros below
[[[0.07822563 0.07548821 0.         0.         0.         0.
   0.         0.         0.         0.        ]]

 [[0.07767532 0.074913   0.         0.         0.         0.
   0.         0.         0.         0.        ]]

 [[0.07743958 0.07460134 0.         0.         0.         0.
   0.         0.         0.         0.        ]]

 [[0.07743079 0.07458631 0.         0.         0.         0.
   0.         0.         0.         0.        ]]]


In [21]:
errorGpu = np.zeros(s)
runTimesGpu = np.zeros(s)
for memNumber in memVec:
    featureMat = functionList.buildFeatureMat(X,memNumber)
    for regularizer in ridgeVecNormal: 
        crossVal = sklearn.model_selection.KFold(n_splits=2)
        cvRun = 0
        for train, test in crossVal.split(featureMat):
            start = time.time()
            w = functionList.solveRrClosedForm(featureMat[train,:],X[train],regularizer)
            end = time.time()
            pred = functionList.predClosedForm(featureMat[test,:],w)
            errorGpu[memVec.index(memNumber), ridgeVec.index(regularizer/l), cvRun] = errorGpu[memVec.index(memNumber), ridgeVec.index(regularizer/l), cvRun] + functionList.findMSE(pred, X[test])
            runTimesGpu[memVec.index(memNumber), ridgeVec.index(regularizer/l), cvRun] = end - start
            cvRun = cvRun + 1

In [24]:
print(runTimesGpu)
print("erros below")
print(errorGpu)

[[[0.14415121 0.08053923 0.         0.         0.         0.
   0.         0.         0.         0.        ]]

 [[0.24266934 0.16192484 0.         0.         0.         0.
   0.         0.         0.         0.        ]]

 [[0.74328494 0.35104179 0.         0.         0.         0.
   0.         0.         0.         0.        ]]

 [[0.54557109 0.46564126 0.         0.         0.         0.
   0.         0.         0.         0.        ]]]
erros below
[[[0.07822571 0.07548829 0.         0.         0.         0.
   0.         0.         0.         0.        ]]

 [[0.07767537 0.07491307 0.         0.         0.         0.
   0.         0.         0.         0.        ]]

 [[0.07743961 0.07460142 0.         0.         0.         0.
   0.         0.         0.         0.        ]]

 [[0.07743081 0.0745864  0.         0.         0.         0.
   0.         0.         0.         0.        ]]]
